<a href="https://colab.research.google.com/github/polyankaglade/ELTeC-eng-loc/blob/main/notebooks/GeoCoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting adresses to unified locations

via API https://developers.arcgis.com/documentation/mapping-apis-and-services/search/services/geocoding-service/

In [ ]:
import requests
import json
import pandas as pd
import csv
from tqdm.auto import tqdm
from functools import cache
tqdm.pandas()

In [ ]:
TOKEN = "AAPK658f0ce0a57a48c29f779dbf7b145437cxbJ6VJ4BC5nlFItF0dHxoTuADRG3xT-FxfquuQYLOzGpAwDdLjeAGF3XZOZR2Cq"

class GeoCoder:
    def __init__(self, token, add_attr: list = []):
        self.__TOKEN = token
        self.__attributes = ['LongLabel', 
                             'Addr_type', 'Type', 
                             # 'PlaceName', 'Place_addr',
                             'StAddr', 'StName', 'AddNum',
                             'City', 'Subregion', 'Region', 
                             'CntryName', 'Country']

        if add_attr:
            self.__attributes.extend(add_attr)

        self.__empty = {k: None for k in self.attributes + ['x', 'y', 'score', 'error']}

        self._url = "https://geocode-api.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates"
        tqdm.pandas()

    @property
    def attributes(self):
        return self.__attributes
    
    @property
    def keys(self):
        return list(self.__empty.keys())
    
    @cache
    def geocode(self, 
                address: str,
                country: str = None,
                max: int = 1):

        payload = {'outFields': ','.join(self.__attributes),
                   'address': address,
                   'maxLocations': max,
                   'f': 'json',
                   'token': self.__TOKEN}

        if country:
            payload['countryCode'] = country

        response = requests.get(self._url, params=payload)

        if response.status_code == 200:
            if len(response.json()['candidates']) == 0:
                return [{'error': 'nothing found'}]
            return response.json()['candidates']

        else:
            return [{'error': response.status_code}]

    
    def code2dict(self, code):
        output = dict(self.__empty.items())
        output['error'] = code.get('error')
        
        if output['error']:
            return output

        output = {**output, **code['attributes']}
        for i in ['x', 'y']:
            output[i] = code['location'][i]
        output['score'] = code['score']
        
        return output

    
    def firstloc2row(self, address, country: str = None):
        return self.code2dict(self.geocode(address, country, max=1)[0])


    def firstloc2tab(self, address, country: str = None):
        result = self.firstloc2row(address, country)
        return '\t'.join(list(result.values()))
                
        
    def process_df(self, df: pd.DataFrame, 
                   col_place: str, col_country: str):
        
        df_geo = df.progress_apply(lambda x: self.firstloc2row(x[col_place], country=x[col_country]), axis=1, result_type='expand')
        df_final = pd.concat([df, df_geo], axis=1)

        return df_final.drop_duplicates()

In [ ]:
coder = GeoCoder(TOKEN)
coder.keys

['LongLabel',
 'Addr_type',
 'Type',
 'StAddr',
 'StName',
 'AddNum',
 'City',
 'Subregion',
 'Region',
 'CntryName',
 'Country',
 'x',
 'y',
 'score',
 'error']

## Authors

In [ ]:
auth_df = pd.read_csv('https://raw.githubusercontent.com/polyankaglade/ELTeC-eng-loc/main/authors/authors_locations_ISO.tsv', sep='\t')
auth_df['place_full'] = auth_df['place'] + ', ' + auth_df['country']
auth_df.head(3)

,author_name,author_id,loc_type,place,place_id,country,country_ISO,place_full
0,"Sinclair, Catherine",Q5052923,place of birth,Edinburgh,Q23436,United Kingdom,GBR,"Edinburgh, United Kingdom"
1,"Sinclair, Catherine",Q5052923,place of death,Kensington,Q288781,United Kingdom,GBR,"Kensington, United Kingdom"
2,"Sinclair, Catherine",Q5052923,country of citizenship,United Kingdom of Great Britain and Ireland,Q174193,United Kingdom of Great Britain and Ireland,GBR,"United Kingdom of Great Britain and Ireland, U..."


In [ ]:
auth_final = coder.process_df(auth_df, col_place='place_full', col_country='country_ISO')
auth_final.to_csv('authors_geo_TOVERIFY.tsv', sep='\t', index=False)

  0%|          | 0/531 [00:00<?, ?it/s]

In [ ]:
auth_final.query('error.notnull()')

,author_name,author_id,loc_type,place,place_id,country,country_ISO,place_full,LongLabel,Addr_type,...,AddNum,City,Subregion,Region,CntryName,Country,x,y,score,error
440,"Conrad, Joseph",Q82925,country of citizenship,Second Polish Republic,Q207272,Second Polish Republic,POL,"Second Polish Republic, Second Polish Republic",None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,nothing found
443,"Conrad, Joseph",Q82925,country of citizenship,Russian Empire,Q34266,Russian Empire,RUS,"Russian Empire, Russian Empire",None,None,...,None,None,None,None,None,None,NaN,NaN,NaN,nothing found


In [ ]:
def update_row_with_dict(df, index, dictionary):
    for key in dictionary.keys():
        df.loc[index, key] = dictionary.get(key)

In [ ]:
update_row_with_dict(auth_final, 440, coder.firstloc2row('Poland'))
update_row_with_dict(auth_final, 443, coder.firstloc2row('Russia'))

In [ ]:
assert len(auth_final.query('error.notnull()')) == 0
auth_final.to_csv('authors_geo_TOVERIFY.tsv', sep='\t', index=False)

## Texts

In [ ]:
output_folder = '/content/drive/MyDrive/Colab Notebooks/HAP-LAP/DH/ELTeC_locations/locations_geo/'

In [ ]:
base_url = 'https://raw.githubusercontent.com/polyankaglade/ELTeC-eng-loc/main/texts/locations_raw/'
meta_df = pd.read_csv('https://raw.githubusercontent.com/polyankaglade/ELTeC-eng-loc/main/metadata.tsv', sep='\t')

filenames = [n+'.xml.csv' for n in meta_df['filename'].tolist()]
len(filenames)

100

In [ ]:
all_lost = []

for i, filename in enumerate(filenames):
    print(i, filename)

    last_country = None
    loc_df = pd.read_csv(base_url+filename)

    outname = filename.replace('.xml.csv', '_geo.tsv')
    outpath = output_folder + outname
    with open(outpath, 'w', encoding='utf8') as csvfile:
        writer = csv.DictWriter(csvfile, delimiter='\t',
                                fieldnames = loc_df.columns.to_list() + coder.keys)
        writer.writeheader()


        for _, row in tqdm(loc_df.iterrows(), total=len(loc_df)):
            if row['ent_type'] == 'GPE':
                addr = coder.firstloc2row(row['Location'])
                last_country = addr.get('CntryName')

                out = {**row.to_dict(), **addr}
                writer.writerow(out)

    lost = pd.read_csv(outpath, sep='\t').query("error.notnull()").copy()
    print(len(lost))
    lost['file'] = outname
    all_lost.append(lost)

0 ENG18410_Sinclair.xml.csv


  0%|          | 0/528 [00:00<?, ?it/s]

4
1 ENG18440_Disraeli.xml.csv


  0%|          | 0/1011 [00:00<?, ?it/s]

4
2 ENG18411_Tupper.xml.csv


...

2
98 ENG19200_Lawrence.xml.csv


  0%|          | 0/308 [00:00<?, ?it/s]

2
99 ENG18400_Trollope.xml.csv


  0%|          | 0/206 [00:00<?, ?it/s]

3


In [ ]:
lost_df = pd.concat(all_lost, axis=0, ignore_index=True)
lost_df.to_csv(output_folder+'errors.tsv', sep='\t', index=False)
len(lost_df)

401

# Filtering

In [ ]:
! pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.7 MB/s eta 0:00:00


In [ ]:
from Levenshtein import distance

In [ ]:
all_df = pd.read_csv('https://raw.githubusercontent.com/polyankaglade/ELTeC-eng-loc/main/texts/all_texts_locations.tsv', sep='\t')
all_df.drop("Unnamed: 0", axis=1, inplace=True)
all_df.head(2)

,Location,ent_type,context,p_no,LongLabel,Addr_type,Type,StAddr,StName,AddNum,...,Subregion,Region,CntryName,Country,x,y,score,error,text_id,author
0,London,GPE,Educated at a small private school in the subu...,0,"London, Greater London, England, GBR",Locality,City,NaN,NaN,NaN,...,Greater London,England,United Kingdom,GBR,-0.127699,51.507408,100.0,NaN,ENG19191,"Hamilton, Cicely"
1,London,GPE,was transferred to a desk in the office of a L...,0,"London, Greater London, England, GBR",Locality,City,NaN,NaN,NaN,...,Greater London,England,United Kingdom,GBR,-0.127699,51.507408,100.0,NaN,ENG19191,"Hamilton, Cicely"


In [ ]:
all_df['Addr_type'].value_counts()

Locality        25259
POI              2601
StreetName         86
PointAddress        2
Postal              1
Name: Addr_type, dtype: int64

In [ ]:
types = all_df['Type'].value_counts()
types[:10]

City                 15452
Country               3188
State or Province     2276
Village               2106
District              1322
Zone                   489
Neighborhood           306
Clothing Store         218
Hotel                  127
Auto Maintenance       125
Name: Type, dtype: int64

In [ ]:
right_types = ['City',
 'Country',
 'State or Province',
 'Village',
 'District',
 'Zone',
 'Neighborhood',
# 'Clothing Store',
 'Hotel',
# 'Auto Maintenance',
 'County',
# 'Specialty Store',
# 'Bar or Pub',
 'Island',
 'Historical Monument',
# 'Medical Clinic',
 'Farm',
 'Church',
 'Forest',
# 'Fitness Center',
# 'Pharmacy',
 'Mountain',
# 'Other Shops and Service',
# 'Sports Center',
 'Stream',
# 'Coffee Shop',
# 'International Food',
# 'Business Facility',
# 'Furniture Store',
 'Park',
 'Parking',
# 'Airport',
# 'Grocery',
# 'Convenience Store',
# 'Footwear',
# 'Residential Area',
 'School',
 'Tourist Attraction',
# 'Rental Cars',
# 'German Food',
# 'Waterfall',
# 'Food and Beverage Shop',
 'Mosque',
# 'Hostel',
 'Lake',
 'Hill',
 'Seamount',
# 'Restaurant',
# 'British Isles Food',
# 'Snacks',
 'Ruin',
# 'Beauty Salon',
# 'Museum',
# 'Shopping Center',
# 'Home Improvement Store',
# 'Consumer Electronics Store',
# 'Indoor Sports',
# 'Repair Services',
# 'Winery',
# 'Sporting Goods Store'
 ]


In [ ]:
all_df['LongLabel'].value_counts()

London, Greater London, England, GBR                      3250
England, GBR                                              1722
Paris, Île-de-France, FRA                                 1405
France                                                     571
Rome, ITA                                                  436
                                                          ... 
New Orleans, LA, USA                                         1
Flyblow Creek, GA, USA                                       1
Amieux, Nanto, Toyama, JPN                                   1
Green Park, West Rand, Gauteng, ZAF                          1
Saint-Vincent-de-Paul, Landes, Nouvelle-Aquitaine, FRA       1
Name: LongLabel, Length: 3270, dtype: int64

In [ ]:
base_url = 'https://raw.githubusercontent.com/polyankaglade/ELTeC-eng-loc/main/texts/GPE_geocoded_v0/'
meta_df = pd.read_csv('https://raw.githubusercontent.com/polyankaglade/ELTeC-eng-loc/main/metadata.tsv', sep='\t')
filenames = meta_df['filename'].tolist()
len(filenames)

100

In [ ]:
def get_levshtein(location_name, long_adress):
    return distance(location_name.strip(), str(long_adress).split(',')[0])

In [ ]:
all_tables = []

for filename in tqdm(filenames):
    loc_df = pd.read_csv(base_url + filename + '_geo.tsv', sep='\t')
    loc_df['filename'] = filename
    loc_df['author'] = meta_df.query(f"filename == '{filename}'")['author-name'].unique()[0]
    loc_df['title'] = meta_df.query(f"filename == '{filename}'")['title'].unique()[0]

    loc_counts = loc_df['LongLabel'].value_counts()
    loc_df['count'] = loc_df['LongLabel'].apply(loc_counts.get)
   
    adr_type_filter = loc_df['Addr_type'].isin(['Locality', 'StreetName', 'POI'])
    type_filter = loc_df['Type'].isin(right_types)

    new_df = loc_df[(adr_type_filter) & (type_filter)].copy()
    new_df['levshtein'] = new_df.apply(lambda x: get_levshtein(x['Location'], x['LongLabel']), axis=1)
    

    all_tables.append(new_df)

final_df = pd.concat(all_tables, axis=0, ignore_index=True)
final_df.to_csv('points_test.tsv', sep='\t')
final_df['levshtein'].value_counts()

  0%|          | 0/100 [00:00<?, ?it/s]

0     22156
1      1872
2       663
4       558
3       229
5       149
7       125
6        92
9        57
8        53
11       38
10       34
14       18
12       12
17       11
13        6
18        4
15        3
16        2
20        2
23        1
30        1
21        1
25        1
Name: levshtein, dtype: int64

In [ ]:
final_df['count'].value_counts()

1.0     3788
2.0     1704
3.0     1113
4.0      920
5.0      775
        ... 
60.0      60
56.0      56
55.0      55
54.0      54
53.0      53
Name: count, Length: 97, dtype: int64

In [ ]:
output_df = final_df[ (final_df['levshtein'] <= 3) & (final_df['count'] >= 5)]
output_df.to_csv('points.csv', sep=',')

In [ ]:
len(all_df) - len(output_df)

10428